In [36]:
# Example of how to get the dataset arrival time form data service blocklatencylog API: 
# based on this query: 
# https://cmsweb.cern.ch/phedex/datasvc/perl/prod/blocklatencylog?
#    dataset=/ReggeGribovPartonMC_EposLHC_pPb_4080_4080_DataBS/pPb816Spring16GS-MB_80X_mcRun2_pA_v4-v2/GEN-SIM
#    &to_node=T2_DE_DESY

DS='/ReggeGribovPartonMC_EposLHC_pPb_4080_4080_DataBS/pPb816Spring16GS-MB_80X_mcRun2_pA_v4-v2/GEN-SIM'
SITE='T2_DE_DESY'
DATASVC_URL='https://cmsweb.cern.ch/phedex/datasvc/json/prod/blocklatencylog'

query=DATASVC_URL+'?dataset='+DS+'&to_node='+SITE
import subprocess
result=subprocess.check_output(['curl','-k','-o','-',query])

In [37]:
# Analyse the result
# Get last_replica times for all dataset blocks
import json
# Get the returned data into a data structure:
logged_data = json.loads(result)

# Data are organized by blocks
# create a hash of the times the block was complete with the block name as a key:
complete_times={}
for block in logged_data['phedex']['block']:
    # print block['destination']['latency']['last_replica']
    for BLK in block['destination']:
        complete_times [block['name']]  = BLK['latency'][0]['last_replica']

#print complete_times   

In [38]:
# Print the time of the last completed block:
from time import gmtime, strftime
def keywithmaxval(d):
    # Recipe borrowed from 
    # https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
    """ a) create a list of the dict's keys and values; 
        b) return the key with the max value"""  
    v=list(d.values())
    k=list(d.keys())
    return k[v.index(max(v))]
    
last_block = keywithmaxval (complete_times)

print 'Last transferred block:\n{0}\narrived on:\n{1}'\
    .format(last_block, strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime(complete_times[last_block])))


Last transferred block:
/ReggeGribovPartonMC_EposLHC_pPb_4080_4080_DataBS/pPb816Spring16GS-MB_80X_mcRun2_pA_v4-v2/GEN-SIM#47f7cf4e-162b-11e7-ba5f-001e67abf094
arrived on:
Wed, 31 May 2017 23:03:36 +0000
